# 第4日の課題

scikit-learnのMLPRegressorで回帰を行います。

## 準備

必要なライブラリを読み込みます。

In [0]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

## データの読み込み

bostonデータは犯罪発生率、部屋数、立地などと不動産価格の関係を示したものです。dataをX、targetをyに格納した後、DESCR属性を表示します。

In [2]:
boston = load_boston()
X = boston.data
y = boston.target
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

### ニューラルネットワークによる回帰

[MLPRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)で回帰関数の学習を行います。

In [3]:
mlpr = MLPRegressor(activation='tanh', max_iter=10000)
mlpr.fit(X, y)

MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=10000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

### 評価

交差確認による[決定係数](http://mathtrain.jp/ketteikeisu)を用いた評価を行います。また、交差確認における分割をランダムにするため、[ShuffleSplit](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html)を使います。

In [4]:
cv = ShuffleSplit(n_splits=3)
scores = cross_val_score(mlpr, X, y, cv=cv, scoring='r2')
print("{0:4.2f} +/- {1:4.2f} ".format(scores.mean(), 2*scores.std()))

0.73 +/- 0.09 


## GridSearch

まず3層構造で、適切な中間層のユニット数を求めます。

In [0]:
param_grid = [
  {'hidden_layer_sizes': [(50,), (100,), (150,), (200,), (300,)]}
 ]

In [6]:
reg = GridSearchCV(mlpr, param_grid, cv=cv)
reg.fit(X, y)
re = reg.cv_results_
for params, mean_score, std_score in zip(re['params'], re['mean_test_score'], re['std_test_score']):
    print("{:.3f} (+/- {:.3f}) for {}".format(mean_score, 2*std_score, params))

0.610 (+/- 0.233) for {'hidden_layer_sizes': (50,)}
0.778 (+/- 0.030) for {'hidden_layer_sizes': (100,)}
0.759 (+/- 0.076) for {'hidden_layer_sizes': (150,)}
0.786 (+/- 0.005) for {'hidden_layer_sizes': (200,)}
0.819 (+/- 0.036) for {'hidden_layer_sizes': (300,)}


ある程度性能の高い300ユニットで階層構造を構築してみます。

In [0]:
param_grid = [
  {'hidden_layer_sizes': [(300,), (150,150), (100,100,100)]}
 ]

In [8]:
reg = GridSearchCV(mlpr, param_grid, cv=cv)
reg.fit(X, y)
re = reg.cv_results_
for params, mean_score, std_score in zip(re['params'], re['mean_test_score'], re['std_test_score']):
    print("{:.3f} (+/- {:.3f}) for {}".format(mean_score, 2*std_score, params))

0.741 (+/- 0.120) for {'hidden_layer_sizes': (300,)}
0.760 (+/- 0.114) for {'hidden_layer_sizes': (150, 150)}
0.732 (+/- 0.112) for {'hidden_layer_sizes': (100, 100, 100)}


多層化の効果は見られますが、層が多ければよいというわけでもなさそうです。

[MLPRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)と[MLPClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)との実装の違いは、そのぞれのマニュアル冒頭に記載してあるように誤差関数です。MLPRegressorは二乗誤差、MLPClassifierはクロスエントロピーを誤差関数として用います。また正解情報は、それぞれ数値、カテゴリである必要があります。